# Regression on Covid-19 incidence

_Akin Kazakci, MINES ParisTech, PSL University_

Input:
- Observed Covid-19 cases (per department)

Output:
- Regression baseline model

The data can be downloaded from https://www.data.gouv.fr/en/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/

In [61]:
import os
import json
import numpy as np
import pandas as pd
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
import xgboost as xgb
from xgboost import plot_importance 
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
import pickle

## Read Data

We will to a simple regregression model per department as a baseline. Data starts at '2020-03-19'. We have 35 points (update with more) per department. This is not much and fitting powerful models is likely to fail. We will use a simple cross-validations scheme, where we shall train on date < '2020-04-08' and test on remaining data.

In France, lockdown has started at '2020-03-16', if we assume that it has shown its effect in observed cases starting 14 days later, we would see that effect starting from '2020-04-01'. Adding one more week for the incubation period of new cases within the new dynamics, puts us at our 'split date' ('2020-04-08') for our cross-validation scheme.

In [68]:
data = pd.read_csv('data.csv', index_col = 0)
data = data[data.date >= '2020-03-25']

In [69]:
# update the split as time goes; make sure we are using same split with later notebooks and models
dates =  data.date
split = '2020-04-20'

In [70]:
data.columns

Index(['code_insee', 'date', 'incid_hosp', 'date_time', 'length_km',
       'movement', 'movement_baseline', 'movement_difference',
       'movement_percent_change', 'density_weighted_movement',
       ...
       'betweenness_centrality-8_16 (t-20)',
       'closeness_centrality-8_16 (t-20)', 'degree_centrality-8_16 (t-20)',
       'eigenvector_centrality-8_16 (t-20)', 'incid_hosp (t-1)',
       'incid_hosp (t-2)', 'incid_hosp (t-3)', 'incid_hosp (t-4)',
       'incid_hosp (t-5)', 'incid_hosp (t-6)'],
      dtype='object', length=335)

# Regression on aggregated data (all departments combined)

In [71]:
data = data[['code_insee','date','incid_hosp','incid_hosp (t-1)',
       'incid_hosp (t-2)', 'incid_hosp (t-3)', 'incid_hosp (t-4)',
       'incid_hosp (t-5)', 'incid_hosp (t-6)' ]]

data = data.sort_values('date', ascending = True).reset_index()
#data.drop(data.date, inplace = True, axis = 1)
data = data.set_index(dates).drop(['date','index'], axis = 1)

data.head()


,code_insee,incid_hosp,incid_hosp (t-1),incid_hosp (t-2),incid_hosp (t-3),incid_hosp (t-4),incid_hosp (t-5),incid_hosp (t-6)
date,,,,,,,,
2020-03-25,48,1.0,146.0,146.0,146.0,2.0,2.0,3.0
2020-03-25,78,80.0,80.0,47.0,47.0,47.0,18.0,18.0
2020-03-25,84,13.0,13.0,80.0,80.0,80.0,47.0,47.0
2020-03-25,84,13.0,80.0,80.0,80.0,47.0,47.0,47.0
2020-03-25,78,80.0,80.0,80.0,47.0,47.0,47.0,18.0


In [72]:
data_train = data.drop('code_insee', axis = 1)

In [73]:
data_train.columns

Index(['incid_hosp', 'incid_hosp (t-1)', 'incid_hosp (t-2)',
       'incid_hosp (t-3)', 'incid_hosp (t-4)', 'incid_hosp (t-5)',
       'incid_hosp (t-6)'],
      dtype='object')

In [74]:
def fit_all(target, dept, split, dates):
    #we put aside everything after the split date, for ultimate validation
    train = dept[dept.index < split].copy()
    val = dept[dept.index >= split].copy()

    xtr, xts = train.drop([target], axis=1), val.drop([target], axis=1)
    print (xtr.shape, xts.shape)
    ytr, yts = train[target].values, val[target].values
    print (ytr.shape, yts.shape)

    # Make sure we are using same time series logic in all pipelines
    tscv = TimeSeriesSplit(30)
    
    # I experimented with various predictors, below I shall use xgboost
    #clf = linear_model.Lasso(alpha=0.5)
    #clf.fit(xtr, ytr)
    
    xgb1 = XGBRegressor()
    # Various hyper-parameters to tune
    parameters = {#'nthread':[4], #when use hyperthread, xgboost may become slower
                  'learning_rate': [.03], #so called `eta` value
                  'max_depth': [3, 5, 7, 10],
                  'subsample': [0.7, 1.0],
                  'colsample_bytree': [0.5,1.0],
                  'n_estimators': [300]}

    xgb_grid = GridSearchCV(xgb1,
                            parameters,
                            cv = tscv,
                            verbose=True)

    xgb_grid.fit(xtr, ytr)
 
    y_preds = xgb_grid.predict(xts)
    print(y_preds.shape)
    error = mean_absolute_error(yts, y_preds)
    print('Mean Error for','target',' is ', error )
    

    dept['preds']=''
    dept['preds'][dept.index < split] = ytr
    dept['preds'][dept.index >= split] =  y_preds
    print(dept.shape)
    return dept, xgb_grid

In [75]:
au, xgb_grid = fit_all('incid_hosp', data_train, split, dates)

(6063, 6) (4016, 6)
(6063,) (4016,)
Fitting 30 folds for each of 16 candidates, totalling 480 fits
[18:25:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[18:25:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[18:25:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:25:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[18:26:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:26:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

...

[18:29:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[18:31:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  7.6min finished


[18:32:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(4016,)
Mean Error for target  is  9.901897794644968
(10079, 8)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [76]:
# save the model
model = xgb_grid.best_estimator_
filename = 'covid-19-auto_regression.sav'
with open(filename, 'wb') as f:
    pickle.dump(model, f)
    f.close()

In [77]:
au.tail()

,incid_hosp,incid_hosp (t-1),incid_hosp (t-2),incid_hosp (t-3),incid_hosp (t-4),incid_hosp (t-5),incid_hosp (t-6),preds
date,,,,,,,,
2020-05-06,6.0,33.0,0.0,16.0,0.0,1.0,1.0,30.6128
2020-05-06,1.0,4.0,5.0,9.0,0.0,10.0,70.0,14.7119
2020-05-06,38.0,0.0,19.0,0.0,1.0,4.0,2.0,12.3025
2020-05-06,27.0,1.0,6.0,38.0,0.0,19.0,0.0,5.17543
2020-05-06,20.0,6.0,33.0,0.0,16.0,0.0,1.0,24.0918


In [ ]:
# The following code is an artefact of an old version, keeping for the moment
#single_data['dept'] = single_data['dept'].apply(lambda x: str(x).encode("utf-8"))
#single_data['dept'] = single_data['dept'].apply(lambda x: int.from_bytes(x, byteorder="big"))

#mBytes = m.encode("utf-8")
#mInt = int.from_bytes(mBytes, byteorder="big")
#mBytes = mInt.to_bytes(((mInt.bit_length() + 7) // 8), byteorder="big")
#m = mBytes.decode("utf-8")

## Comparing with indvidual models

# TODO
## Must revise & clean the old notebook (single models, 8_auto_regress_per_dept)

Raw score (MAE) is better than the mean of MAE for all departments. However, I would like to be able to compare department per department. This requires finding 'dept' indices for the rows of au (augmented single data matrix).

In [190]:
#au = au.loc[(au[['incid_hosp','t-1','t-2','t-3','t-4','t-5','t-6']].sum(axis=0) != 0)]
single_data.head(5)
#au[au.duplicated()]

,incid_hosp,t-1,t-2,t-3,t-4,t-5,t-6,dept,preds,error
date,,,,,,,,,,
2020-03-25,12,11.0,14.0,3.0,3.0,0.0,1.0,12337,12,0
2020-03-26,13,12.0,11.0,14.0,3.0,3.0,0.0,12337,13,0
2020-03-27,15,13.0,12.0,11.0,14.0,3.0,3.0,12337,15,0
2020-03-28,7,15.0,13.0,12.0,11.0,14.0,3.0,12337,7,0
2020-03-29,10,7.0,15.0,13.0,12.0,11.0,14.0,12337,10,0


In [174]:
true_labels = single_data[single_data.index >= split]['incid_hosp']
print(true_labels.shape)
predicted = single_data[single_data.index >= split]['preds']


print(predicted.shape)
sum_error = np.sum(np.abs(true_labels-predicted))
print(len(predicted))
print(round(mean_absolute_error(true_labels,predicted),2))
print(true_labels.sum())

(1515,)
(1515,)
1515
9.66
30332


In [177]:
single_data['error'] = np.abs( single_data['incid_hosp'] - single_data['preds'])

In [191]:
gr = single_data[single_data.index >= split].groupby('dept', as_index = False)
for n,g in gr:
    mBytes = n.to_bytes(((n.bit_length() + 7) // 8), byteorder="big")
    m = mBytes.decode("utf-8")
    print (m, round(g['error'].mean(),2))

01 7.32
02 18.88
03 2.38
04 3.66
05 2.44
06 7.58
07 4.92
08 6.04
09 2.32
10 23.21
11 4.35
12 2.82
13 32.46
14 3.8
15 2.15
16 2.3
17 4.75
18 5.11
19 2.29
21 14.25
22 3.34
23 3.97
24 4.56
25 11.74
26 4.7
27 8.67
28 9.11
29 3.12
2A 2.17
2B 4.26
30 8.91
31 7.18
32 2.16
33 6.61
34 9.27
35 5.17
36 4.81
37 5.88
38 8.55
39 4.3
40 3.05
41 4.3
42 10.15
43 3.75
44 7.02
45 6.03
46 4.97
47 2.01
48 2.28
49 5.69
50 3.77
51 12.03
52 5.87
53 4.61
54 13.05
55 4.73
56 6.4
57 30.48
58 3.29
59 20.8
60 16.38
61 3.1
62 15.1
63 2.29
64 2.86
65 3.42
66 2.87
67 24.64
68 28.27
69 24.24
70 5.01
71 9.8
72 4.69
73 4.38
74 8.2
75 53.24
76 7.33
77 25.82
78 21.38
79 2.25
80 9.4
81 2.57
82 2.03
83 11.05
84 3.2
85 4.83
86 2.97
87 2.8
88 9.7
89 5.77
90 6.63
91 41.17
92 43.82
93 49.61
94 51.53
95 25.04
971 2.22
972 2.64
973 2.44
974 2.9
976 2.21


TODO: Put in the same format as the above scores table.